In [1]:
import sys

sys.path.insert(0,'..')

from utils.comet_definition import Comet0
import pandas as pd
import ast
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


print("model loading ...")
comet = Comet0('../../lib/comet-atomic-2020/models/comet_atomic2020_bart/comet-atomic_2020_BART')
comet.model.zero_grad()
print("model loaded")


model loading ...
model loaded


In [2]:
queries = []
head = "PersonX went to the Hospital"
rel = "xWant"
query = "{} {} ".format(head, rel)
queries.append(query)
print(queries)
results = comet.generate(queries, decode_method="top_k", num_generate=10)
print(results)

['PersonX went to the Hospital xWant ']
[[' to get a prescription from the doctor.', ' to be taken to the hospital by a doctor.', ' to be treated at the hospital', ' to be treated at the hospital', ' to get better', ' to get a prescription for pain killers.', ' to be taken to the hospital by the doctor.', ' to be taken to the hospital by a doctor.', ' to be taken to the hospital by a doctor.', ' to be taken to the hospital by a doctor.']]


In [3]:
roc_stories_triples_csv = '../../generated/knowledge-triple-extraction/ROCStories_resolved_with_knowledge_triples.csv'
roc_stories_triples_df = pd.read_csv(roc_stories_triples_csv, sep='\t', header=0)

In [4]:
roc_stories_triples_df = roc_stories_triples_df.iloc[:1]

In [5]:
del roc_stories_triples_df['Unnamed: 0']
del roc_stories_triples_df['Unnamed: 0.1']
del roc_stories_triples_df['Unnamed: 0.1.1']
del roc_stories_triples_df['Unnamed: 0.1.1.1']

In [6]:
del roc_stories_triples_df['srl_r1']

In [7]:
# roc_stories_triples_df

In [8]:
# xRels = ["xWant", "xNeed", "xEffect"]
# oRels = ["oWant", "oEffect"]

In [9]:
from itertools import chain
import multiprocessing as mp
from tqdm import tqdm

In [10]:
# %%time
# for n in range(1,6): 
#     print(f"n = {n}")
#     new_events_rows = []
#     for index, row in tqdm(roc_stories_triples_df.iterrows()):
#         queries = []
#         relations = []
#         triples_res = []
#         triples = ast.literal_eval(row[f'triples{n}'])  
#         for triple in triples:
#             comet_head = ' '.join(triple)
# #             print(comet_head)
#             for rel in xRels:
#                 query = "{} {} ".format(comet_head, rel)
#                 queries.append(query)
#                 relations.append(rel)
#                 triples_res.append(triple[0])
#             for rel in oRels:
#                 query = "{} {} ".format(comet_head, rel)
#                 queries.append(query)
#                 relations.append(rel)
#                 triples_res.append(triple[2])
#         results = comet.generate(queries, decode_method="beam", num_generate=5)
#         results = [list(filter(lambda x: x.strip() != 'none', r)) for r in results]
#         new_events = list(zip(triples_res,relations, results))
# #         print(new_events)
#         new_events_rows.append(new_events)
#     roc_stories_triples_df[f'new_triples{n}'] = new_events_rows
    

In [11]:
forward_rels = ["xWant", "isBefore", "xEffect"]
backward_rels = ["xIntent", "isAfter", "xNeed"]

In [12]:
%%time
result_df = pd.DataFrame()
story_ids = []
sentence_ids = []
results_final = []
for n in range(1,6):
    print(f"n = {n}")
    tmp_df = pd.DataFrame()
    new_events_rows = []
    queries = []
    relations = []
    triples_res = []

    for index, row in tqdm(roc_stories_triples_df.iterrows()):
        story_id = row['storyid']
        sentence_id = f'sentence_{n}'
        triples = ast.literal_eval(row[f'triples{n}']) 
        for idx, triple in enumerate(triples):
            comet_head = ' '.join(triple)
#             print(comet_head)
            if n < 5:
                for rel in forward_rels:
                    query = "{} {} ".format(comet_head, rel)
                    queries.append(query)
                    relations.append(rel)
                    triples_res.append(triple)
                    # for pivot
                    story_ids.append(story_id)
                    sentence_ids.append(sentence_id)
            if n > 1:
                for rel in backward_rels:
                    query = "{} {} ".format(comet_head, rel)
                    queries.append(query)
                    relations.append(rel)
                    triples_res.append(triple)
                    # for pivot
                    story_ids.append(story_id)
                    sentence_ids.append(sentence_id)
    
    print('Genrating events...')
    results = comet.generate(queries, decode_method="beam", num_generate=10)
    results = [list(filter(lambda x: x.strip() != 'none', r)) for r in results]
    new_events = [list(a) for a in zip(triples_res,relations, results)]
    
    print(new_events)
    results_final += new_events

    print(len(results_final))
    
result_df['storyid'] = story_ids
result_df['sentence_id'] = sentence_ids
result_df[f'new_events'] = results_final
    
#     result_df.to_csv()

n = 1


1it [00:00, 1732.47it/s]

Genrating events...


[[["Dan 's parents", 'were', 'overweight'], 'xWant', [' to buy new clothes', ' to buy healthy food', ' to eat less', ' to lose weight', ' to eat healthy', ' to eat less food', ' to eat healthier', ' to eat healthy food', ' to eat less.', ' to lose weight.', ' to work out more', ' to buy healthy foods', ' to work out', ' to gain weight', ' to be healthy', ' to be healthier', ' to eat more', ' to get healthy', ' to exercise', ' eat less']], [["Dan 's parents", 'were', 'overweight'], 'isBefore', [" Dan 's parents go on a diet", ' Dan goes to the gym to lose weight', ' Dan and his parents go on a diet', ' Dan goes to the gym with his parents', " Dan 's parents went on a diet", " Dan 's parents go to the gym", ' Dan and his parents go to the gym', ' PersonX eats a lot of food.', ' PersonX eats a lot of junk food', " Dan 's parents go to the doctor", " Dan 's parents went to the doctor", ' Dan buys new clothes for his parents', " Dan 's parents went to the gym", ' Dan eats a lot of junk food

1it [00:00, 2058.05it/s]

...

Genrating events...


[[["Dan 's parents", 'decided', 'to make a change'], 'xWant', [' to start a new life', ' to get a new job', ' to have a better life', ' to find a new job', ' to make a change themselves', ' to see how it goes', ' to make a good impression', ' to see how they do', ' to see what they think', ' to start a new business', ' to make a new life', ' to go back to work', ' to have a new life', ' to try something new', ' to go back to school', ' to make a change', ' to go to the gym', ' to enjoy the change', ' to go on vacation', ' to make a plan']], [["Dan 's parents", 'decided', 'to make a change'], 'isBefore', [' Dan and his parents move to a new city', ' Dan and his parents move to a new house', ' Dan and his parents move to a new town', ' Dan and his parents move to a new home', ' Dan and his parents get a new job', ' Dan and his parents go to a new restaurant', " Dan 's parents get a new job", ' Dan and his parents go to a new school', ' Dan and his parents move to a new place', ' Dan and 

1it [00:00, 909.24it/s]

Genrating events...


[[["Dan 's parents", 'got', "Dan 's parents and Dan"], 'xIntent', [' to spend time with them', ' to be with their parents', ' to be a good parent', ' to have a good time', ' to take care of them', ' to be with their family', ' to spend time with family', ' to be with his parents', ' to be a good friend', ' to spend time with him', ' to be close to them', ' to spend time together', ' to be with them', ' to show their love', ' to be with family', ' to have a family', ' to talk to them', ' to be helpful', ' to be nice', ' to be together']], [["Dan 's parents", 'got', "Dan 's parents and Dan"], 'isAfter', [' PersonX asks PersonX to marry them.', ' PersonX asks PersonY to marry them.', " PersonX calls PersonX's parents.", ' PersonX wants to go to the movies.', ' Dan and his parents go to the movie theater', " PersonX calls PersonY's parents.", ' PersonX asks PersonX to marry them', ' Dan and his parents go to the movies', " PersonX calls PersonX's parents", ' PersonX asks PersonY to marry t

In [13]:
# result_df

In [14]:
tmp = result_df.groupby(['storyid', 'sentence_id']).new_events.apply(list).to_frame().reset_index()

In [15]:
res = tmp.pivot(index='storyid', columns='sentence_id', values='new_events').reset_index()

In [16]:
res.columns.name = None

In [17]:
res

storyid  \
0  9a51198e-96f1-42c3-b09d-a3e1e067d803   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          sentence_1  \
0  [[[Dan 's parents, were, overweight], xWant, [ to buy new clothes,  to buy healthy food,  to eat less,  to lose weight,  to eat healthy,  to eat less food,  to eat healthier,  to eat healthy food,  to eat less.,  to lose weight.,  to work out more,  to buy healthy foods,  to work out,  to gain weight,  to be healthy,  to be healthier,  to eat more,  to get healthy,  to exercise,  eat less]], [[Dan 's parents, were, overweight], isBefore, [ Dan 's parents go on a diet,  Dan goes to the gym to lose weight,  Dan and his parents go on a diet,  Dan goes to the gym with his parents,  Dan 's parents went on a diet,  Dan 's parents go to the gym,  Dan and his parents go to the gym,  PersonX eats a lot of food.,  PersonX eats a lot of junk food,  Dan 's parents go to the doctor,  Dan 's parents went to the doctor,  Dan buys new clothes for his parents,  Dan 's parents went to the gym,  Dan eats a lot of junk food,  Dan's parents go to the gym,  PersonX eats a lot of food,  Dan 's parents lose weight,  Dan goes to the gym,  Dan gets a new job]], [[Dan 's parents, were, overweight], xEffect, [ goes to the doctor,  goes to the gym,  has to eat less,  has to lose weight,  gets a restraining order,  gets teased by parents,  has to exercise more,  has to eat more,  has to buy food,  to parents to eat,  eats too much,  eat too much,  goes to doctor,  goes to gym,  has to exercise,  loses weight,  gain weight,  gets fat,  eats less]]]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [18]:
def forward_filter(triple):
    return triple[1] in forward_rels

In [19]:
def backward_filter(triple):
    return triple[1] in backward_rels

In [20]:
def get_forward_triples(triples, index):
    forward_events = []
    for triple in triples:
        if triple[1] == forward_rels[index]:
            forward_events.append(triple)
    return forward_events

In [21]:
def get_backward_events(triples, index):
    backward_events = []
    for triple in triples:
        if triple[1] == backward_rels[index]:
            backward_events += triple[2]
    return [a.strip() for a in backward_events]

In [22]:
n_rels = len(forward_rels)
for idx, row in tqdm(res.iterrows()):
    print(f'row {idx}')
    for f in range(1,5):
        print(f'row {idx}, sentence {f}')
        triples_f = row[f'sentence_{f}']
        forward_events = list(filter(forward_filter, triples_f))
        backward_events = []
        for b in range(f+1,6):
            triples_b = row[f'sentence_{b}']
            backward_events += list(filter(backward_filter, triples_b))
        pruned_triples_f = []
#         print(backward_events)
        for rel in range(n_rels):
            print(f'row {idx}, sentence {f}, relation {forward_rels[rel]}')
            ft = get_forward_triples(forward_events, rel)
#             print(ft)
            be = get_backward_events(backward_events, rel)
#             print(be)
            new_ft = []
            for triple in ft:
#                 print(triple)
#                 print(triple)
#                 print(type(triple[2]))
                triple_copy = [a for a in triple]
                triple_copy[2] = [event.strip() for event in triple[2] if event.strip() in be]
                print(triple_copy)
                if triple_copy[2]:
                    new_ft.append(triple_copy)
#             print(new_ft)
            pruned_triples_f += ft

1it [00:00, 313.29it/s]

row 0
row 0, sentence 1
row 0, sentence 1, relation xWant
[["Dan 's parents", 'were', 'overweight'], 'xWant', ['to eat less', 'to lose weight', 'to eat healthy', 'to eat healthier', 'to gain weight', 'to be healthy', 'to be healthier', 'to eat more', 'to get healthy', 'to exercise', 'eat less']]
row 0, sentence 1, relation isBefore
[["Dan 's parents", 'were', 'overweight'], 'isBefore', ["Dan 's parents go on a diet", 'Dan goes to the gym with his parents', "Dan 's parents went on a diet", "Dan 's parents go to the gym", 'PersonX eats a lot of food.', 'PersonX eats a lot of junk food', "Dan 's parents go to the doctor", "Dan 's parents went to the doctor", "Dan 's parents went to the gym", 'PersonX eats a lot of food', "Dan 's parents lose weight", 'Dan goes to the gym', 'Dan gets a new job']]
row 0, sentence 1, relation xEffect
[["Dan 's parents", 'were', 'overweight'], 'xEffect', ['eat too much']]
row 0, sentence 2
row 0, sentence 2, relation xWant
[['Dan', 'was', 'overweight'], 'xWan